# SOLUTIONS FOR PCA #11

## Second-order solutions to Burgers' equation

This is an implementation of the solution to Burgers' equation ($u_t + u u_x = 0$) using the 2nd-order finite volume solution from Chapter 6 of Zingale's hydro lecture notes. using the two different slope limiters that we implemented for the last in-class assignment.

I've added several different initial conditions types, including the sine and rarefaction wave from Zingale's examples for Chapter 6, but also the Gaussian and tophat ICs from previous pre-class/in-class assignments.  They have different behavior that's interesting in its own way.

I've also implemented different boundary condition types - periodic and Dirichlet.  The periodic BCs are straightforward; Dirichlet just copies the zone immediately inside of the boundary into the boundary conditions.


## User-set parameters

The user should set all of the parameters in this cell, which are described immediately above them.

In [ ]:
# grid size (not including ghost zones)
N = 128               

# CFL (must be <= 1.0)
C = 0.8

# number of periods - could be anything > 0.  
N_periods = 0.3

# Initial condition: could be 'Gaussian' or 'tophat' or 'sine' or 'rarefaction'
IC_type = 'sine'

# Flip direction of the initial conditions from + to - values.  True or False.
invert_ICs = False

# Boundary condition: could be 'periodic' or 'Dirichlet' 
BC_type = 'Dirichlet'  

# advection method: always 2nd order, but could have minmod limiter ('2oa_minmod')
# or MCD limiter ('2oa_mcd')
adv_method  = '2oa_mcd' 

# Save a copy of the auto-named plot to disk?  True or False
save_figure = False 

## And now, the code!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# make a title (for plots) and file name (for writing out)

mytitle = "Burgers' equation: " + "Ng = " + str(N) + ", Np = " + str(N_periods) + \
",\n IC = " + IC_type + ", method = " + adv_method

myfilename = "Burgers_Ng_" + str(N) + "_Np_" + str(N_periods) + "_" + IC_type + "_" + adv_method + ".png"

print("TITLE:\n",mytitle)
print("\nFILE NAME:  ", myfilename)

# Grid dx
dx = 1.0/N


### Define all of the functions we're going to use

If this weren't in a Jupyter notebook all of these would be in their own file, summoned from a driver script as necessary.  Honestly, it'd be cleaner to package it all in an object and then run it, but for the purposes of understanding what's going on the notebook is more convenient.

In [ ]:
def ICs(N=100,IC_type='tophat',invert=False):
    '''
    Initial condition generator.  Inputs are the desired number of grid 
    points (N) and initial conditions type (IC_type, options are 'tophat', 
    'Gaussian', 'sine', 'rarefaction').  Assumes that it's a second-order
    method with two ghost zones. 
    
    Outputs are an array of positions (mostly useful for plotting) and
    the array values that are going to be advected.
    '''

    # the method we're implementing needs two ghost zones!
    ghost_zones = 2    
    
    # 'u' array starts out zero everywhere.
    u = np.zeros(N+2*ghost_zones)
    
    dx=1.0/N
    
    # set up positions, including ghost zones, and also a step of 0.5*dx to make sure the points are at
    # the middle of the cell.
    x = np.linspace(0.0-dx*ghost_zones,1.0+dx*ghost_zones,num=N+2*ghost_zones,endpoint=False)+0.5*dx
    
    if IC_type=='tophat':  # top hat ICs
        
        # make a filter array that's True for the region we want 
        # to be 1, then set the array.
        filter_array = np.logical_and( x>0.35 , x<0.65)
        u[filter_array]=1.0

    elif IC_type=='Gaussian':  # Gaussian ICs
        sigma = 0.1
        u = np.exp(-(x-0.5)**2/(2*sigma*sigma))
        
    elif IC_type=='sine':     # sine wave at the center of the box
        
        u=1.0+0.5*np.sin( 2.0*np.pi*(x-1./3.)/(1./3.)  )
        
        # make a filter array that's True for the region we want 
        # to modify, then set the array.
        filter_array = np.logical_or( x<0.3333 , x>0.6667)
        u[filter_array]=1.0

    elif IC_type=='rarefaction':  # rarefaction wave
        u[x<=0.5]=1.0
        u[x>0.5]=2.0
        
    else: # if the user has a typo, announce it and quit.
        print("You didn't put in a correct IC type option:", IC_type)
        exit()
    
    if invert==True:
        u = -u
    
    return x, u

In [ ]:
def apply_BCs(array, bc_method='periodic'):
    '''
    Copies ghost zones for array.  Assume second order method, 
    so copy two ghost zones.
    '''
        
    if bc_method == 'periodic': 
        # copy the inner two "active" zones from the other side of the array 
        # into the ghost zones
        array[0:2]=np.copy(array[-4:-2])
        array[-2:]=np.copy(array[2:4])
        
    elif bc_method == 'Dirichlet':
        # Copy only the endmost 'active' zone from the same side of the array
        # into the ghost zones
        array[0:2]=np.copy(array[2])
        array[-2:]=np.copy(array[-3])           
            
    else: # user has made a typo. tell them and quit.
        print("THIS BOUNDARY CONDITION NOT IMPLEMENTED:", bc_method)
        exit()


In [ ]:
def minmod(arr,dx):
    '''
    minmod limiter - return 1st derivative of input array, 
    assume array being input has 1 extra zone on each side, so 
    the returned da/dx is 2 cells shorter than input array.
    '''
    
    dadx = np.zeros(arr.size-2)
    
    # I'm doing a loop here; this could probably be replaced by something clever using numpy,
    # which would almost certainly speed it up substantially compared to this loop.
    # maybe next time I teach the class.
    for i in range(dadx.size):
        # the array indices here are all +1 more than they naively should be b/c of 
        # the ghost zones in the input array (da/dx is 2 cells shorter than arr)
        a = arr[i+1]-arr[i]
        b = arr[i+2]-arr[i+1]
        
        if np.abs(a) < np.abs(b) and a*b > 0:
            
            dadx[i] = a/dx

        elif np.abs(b) < np.abs(a) and a*b > 0:
        
            dadx[i] = b/dx

        else:
            dadx[i] = 0.0
            
    return dadx

In [ ]:
def mcd(arr,dx):
    '''
    monotonized central difference limiter (MCD) - return 1st 
    derivative of input array, assume array being input has 1 
    extra zone on each side, so the returned da/dx is 2 cells 
    shorter than input array.
    '''
    
    dadx = np.zeros(arr.size-2)
    
    # I'm doing a loop here; this could probably be replaced by something clever using numpy,
    # which would almost certainly speed it up substantially compared to this loop.
    # maybe next time I teach the class.
    for i in range(dadx.size):
        # the array indices here are all +1 more than they naively should be b/c of 
        # the ghost zones in the input array (da/dx is 2 cells shorter than arr)
        a = arr[i+1]-arr[i]
        b = arr[i+2]-arr[i+1]
        c = arr[i+2]-arr[i]
        
        if a*b > 0.0:
            dadx[i] = np.fmin( np.fabs(c)/2.0, np.fmin( 2.0*np.fabs(b), 2.0*np.fabs(a) ))*np.sign(c)/dx
            
        else:
            dadx[i] = 0.0
            
    return dadx

In [ ]:
def advect(u,dt,dx,method='2oa_minmod'):
    '''
    Advection routine.  Evolves Burgers' equation forward one timestep for 2nd order 
    finite volume method (including either the minmod or mcd limiter, as specified by
    the user).  
    
    Inputs are:
    
    u = array that will be evolved
    dt = time step
    dx = grid spacing
    method = numerical method (2nd order with limiter: '2ou_minmod' or '2ou_mcd')
    
    Nothing is returned; the array 'u' is modified in place, except for the ghost
    zones (which are modified elsewhere).
    '''


    '''
    Get du/dx using either the minmod or mcd slope limiter.  The returned du/dx will be
    2 cells smaller than the array u, which is assumed to have two ghost zones.
    That means that it only gives the derivative for the first ghost zone outside
    of the domain, which is all that we need to calculate the left and right states for
    the advection.
    '''
    if method=='2oa_minmod':
        dudx = minmod(u,dx)
    elif method=='2oa_mcd':
        dudx = mcd(u,dx)
    else:
        print("WHOAH THERE PARDNER, WRONG METHOD SPECIFIED:", method)
        exit()
    
    
    '''
    Estimate the state of u on the i+1/2 face of cell using info from cell i (the "Left" state).
    Note that we're only getting these states for cells 2...n-3 (i.e., skipping the 
    ghost zones).  That said, the ul array will have one extra cell compared to the 
    number of active zones in the grid (i.e., non-ghost zones) because for N cells
    we need N+1 faces.  
    '''
    ul = u[1:-2]+0.5*dx*(1.0-(dt/dx)*u[1:-2])*dudx[:-1]
  
    '''
    Estimate the state of u in the i+1/2 face of the cell using the info from cell 
    i (the "Right" state).  The ur array also has one extra cell, for the same reason as 
    the ul array.
    '''
    ur = u[2:-1]-0.5*dx*(1.0+(dt/dx)*u[2:-1])*dudx[1:]
    
    # Estimate shock speed (Zingale 6.13)
    S = 0.5*(ul+ur)

    '''
    Now set the value of u on each face, based on whether or not we have a shock (from array S).
    This array, like ul and ur, will have one more cell than the number of active cells, for the
    same reason (N cells need N+1 faces).
    '''
    
    # Create an array of zeros to store everything in
    us = np.zeros_like(S)
    
    '''
    Now we apply the logic from Zingale equations 6.14 and 6.15.  Loop over the array to do so.
    This probably can be done using numpy cleverness, but it's late and I'm tired.
    '''
    for i in range(S.size):
        if ul[i] > ur[i]:  # if this is true there's a shock (Zingale Eqtn. 6.14)
            if S[i] > 0.0:
                us[i] = ul[i]
            elif S[i] < 0.0:
                us[i] = ur[i]
            else:
                us[i]=0.0
        else:  # otherwise, there's no shock (Zingale Eqtn. 6.15)
            if ul[i] > 0.0:
                us[i] = ul[i] 
            elif ur[i] < 0.0:
                us[i] = ur[i]
            else:
                us[i]=0.0
    
    '''
    Once we're done setting the state values, we do the conservative update to array u 
    (Zingale 6.17).  The flux is defined as F = 0.5*u^2 (Zingale 6.16), so we're calculating the update for
    cell i using the cell faces i-1/2 and i+1/2:
    
    u_i^{n+1} = u_i^n - (dt/dx)*( F_{i+1/2}^{n+1/2} - F_{i-1/2}^{n+1/2} )
    
    Note (1): I'm ONLY setting the active (non-ghost) zones for the u array , which are u[2:-2], 
    and since I've stored the left and right face values in the array "us" that means the left
    sides (i-1/2 values) of the active zones are in us[:-1] and the right sides (i+1/2 values)
    are in us[1:] (using standard numpy array slicing).
    
    Note (2): My signs on the RHS of this expression are oppositee of what Zingale uses in Eqt. 6.17 
    because I'm staying consistent with the expressions from elsewhere in the book.
    '''
    u[2:-2] = u[2:-2] - (dt/dx)*(0.5*(us[1:]**2)-0.5*(us[:-1]**2))


In [ ]:
def makeplot(x,a_orig,a_now,this_title='Output Information',filename='output.png',savefig=True):
    '''
    Makes a plot and (optionally) saves it to a file.  Inputs are:
    x = position array
    a_orig = original set of a values (initial conditions)
    a_now = current time set of a values
    this_title = your title
    filename = desired filename
    savefig = Boolean for whether or not you want your figure saved to a file.
    
    Output is a plot displayed to screen and (optionally) saved to file.
    '''
    plt.plot(x,a_orig,'k-')
    plt.plot(x,a_now,'r--')
    plt.xlabel('x')
    plt.ylabel('u(x)')
    plt.title(this_title)
    if savefig==True:
        plt.savefig(filename,dpi=400)


## Now run things!

If I weren't writing this in a Jupyter notebook, the following code would all be in a driver routine.  But, notebooks let us inline our pretty, pretty plots, so that's why we're using them here.

In [ ]:
# set up initial conditions arrays
x, u = ICs(N,IC_type,invert=invert_ICs)

# make a copy of our initial conditions for plotting purposes later
u_original = np.copy(u) 

# make a quick plot of the initial conditions so we can admire them
plt.plot(x,u_original,'b-')
plt.xlabel('x')
plt.ylabel('u(x)')
plt.title('Initial conditions')

In [ ]:
# This cell evolves our simulation until the end time, using a timestep dt that we calculate as we go.

# final time (take abs. magnitude of velocity or else it may be negative)
# use maximum velocity magnitude to estimate the final time.
T_final = N_periods/np.max(np.abs(u))

# initial time, which is always zero as per tradition.
t=0.0

# time step is the minimum of all dx/u_i, taking the absolute
# value of u_i so we always move forward in time.
dt = C*np.min(dx/np.abs(u))

# loop until we're done.
# the -dt in the while() is because we add 
# the dt after we do the evolution.
# otherwise, things aren't quite right in a logical
# sense.
while(t < T_final-dt):
    
    # calculate timestep
    dt = C*np.min(dx/np.abs(u))
    
    # advect u by one timestep dt using the chosen method
    advect(u,dt,dx,method=adv_method)
    
    # apply boundary conditions for next step
    apply_BCs(u,bc_method=BC_type)
    
    # make a plot, just for fun (not necessary, but interesting.)
    plt.plot(x,u,linewidth=2,alpha=0.5)
    
    t+=dt


In [ ]:
# make our plot and (optionally) save it.
makeplot(x,u_original,u,mytitle,myfilename,savefig=save_figure)